# Các cách làm giàu cho Audio

In [1]:
from audiomentations import AddBackgroundNoise, PolarityInversion, Compose,AddGaussianNoise, AddGaussianSNR, AddShortNoises, Mp3Compression, Compose
import soundfile as sf
import numpy as np
import librosa
import audiomentations

In [5]:
def add_backgroundNoise(path_input_audio,path_sound_noise_add = "/home/pdnguyen/Audio_Augmentation/noise"):
    """
    Input: Path âm thanh gốc + path thư mục chứa noise muốn thêm vào
    Output: Âm thanh làm giàu dưới dạng Array
    """
    noise_transform = PolarityInversion()  # Define your noise transform here
    augment = Compose([
        AddBackgroundNoise(sounds_path=path_sound_noise_add, #path noise 
                            noise_rms="relative",
                            min_snr_in_db=3.0, # biên độ lọc dưới của noise
                            max_snr_in_db=30.0, #biên độ lọc trên của nosie
                            p=1.0),
        noise_transform  # Apply the noise transform after AddBackgroundNoise
    ])
    audio_input,_ = sf.read(path_input_audio)
    augmented_sound = augment(audio_input, sample_rate=16000)
    return augmented_sound

In [15]:
def add_gaussianNoise(path_input_audio):
    """
    Input: Path âm thanh gốc
    Output: Âm thanh làm giàu nhiễu + mức độ nhiễu kiểm soát theo độ lệch chuẩn (hình chuông) trả về dưới dạng Array => giống nghe đài
    Tham khảo nhiễu color và Gauss: https://cmtext.indiana.edu/acoustics/chapter1_waves3.php#:~:text=GAUSSIAN%20NOISE&text=Named%20after%20mathematician%2Fphysicist%20Carl,as%20lacking%20in%20bass%20frequencies.
    """
    transform = AddGaussianNoise(
        min_amplitude=0.001,
        max_amplitude=0.015,
        p=1.0
    )
    audio_input,_ = sf.read(path_input_audio)
    augmented_sound = transform(audio_input,sample_rate=16000)
    return augmented_sound


In [ ]:
def add_gaussianNoiseSNR(path_input_audio):
    """
    Input: Path âm thanh gốc
    Output: Âm thanh làm giàu nhiễu gauss SNR - mức độ nhiễu phụ thuộc vào tỷ lệ công suất signal và noise
    """
    transform = AddGaussianSNR(
    min_snr_db=5.0,
    max_snr_db=40.0,
    p=1.0
    )
    audio_input,_ = sf.read(path_input_audio)
    augmented_sound = transform(audio_input,sample_rate=16000)
    return augmented_sound

In [75]:
def add_white_noise(path_input_audio, noise_level=0.01):
    """
    Input: Path âm thanh origin
    Output: Âm thanh thêm nhiễu trắng (mô phỏng âm thanh nhiễu từ đài)
    Tham khảo: https://cmtext.indiana.edu/acoustics/chapter1_waves3.php#:~:text=GAUSSIAN%20NOISE&text=Named%20after%20mathematician%2Fphysicist%20Carl,as%20lacking%20in%20bass%20frequencies.
    """
    audio,_ = sf.read(path_input_audio)
    return audio + noise_level * np.random.randn(len(audio))


def zip_mp3(path_input_audio):
    """
    Input: Path âm thanh gốc
    Output: Âm thanh được nén xuống mp3 => giảm chất lượng audio đi
    """
    tran = Mp3Compression(min_bitrate=8, max_bitrate=10, p=1.0)
    audio, sr = librosa.load(path_input_audio, sr=None)
    audio_aug = tran(audio,sample_rate=16000)
    return audio_aug 


In [27]:
def add_short_noise(audio_path, noise_dir):
    """
    Input: Path âm thanh gốc
    Output: Âm thanh + các nhịp âm thanh short cách nhau một khoảng random
    """
    # Load the audio file
    audio, sr = librosa.load(audio_path, sr=None)
    
    # Create the AddShortNoises augmenter
    augmenter = Compose([
        AddShortNoises(
            sounds_path=noise_dir,  # Pass the directory path directly
            min_snr_in_db=5,
            max_snr_in_db=30,
            min_time_between_sounds=0.0005,#tgian tạm dừng tối thiểu của noise
            max_time_between_sounds=0.001,#tgian tạm dừng tối đa của noise
            p=1.0
        )
    ])
    
    # Apply the augmentation
    augmented_audio = augmenter(samples=audio, sample_rate=sr)
    
    return augmented_audio   

In [26]:
import numpy
path_input_audio = "/home/pdnguyen/Audio_Augmentation/sounds_file/0_1ch_na.wav"
sf.write("check222.wav",audio_aug, 16000)